In [1]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
from nltk import word_tokenize
from functools import lru_cache
import re
import unicodedata
import sys
from collections import Counter
import nltk
nltk.download('brown')
from nltk.corpus import brown
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Lexical_simp'



Mounted at /content/drive
/content/drive/MyDrive/Lexical_simp


In [3]:
%cd '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/'
!pip install -e . 
from muss.mining.training import get_bart_kwargs, get_score_rows
from muss.resources.prepare import prepare_wikilarge_detokenized, prepare_asset
from muss.resources.datasets import create_smaller_dataset

from muss.fairseq.main import fairseq_train_and_evaluate_with_parametrization
from muss.mining.training import get_score_rows #get_bart_kwargs

#########################
import re
import shutil
import os 

from cachier import cachier
from easse.cli import evaluate_system_output
from easse.utils.constants import TEST_SETS_PATHS
import torch
from tqdm import tqdm

from muss.resources.paths import get_data_filepath, MODELS_DIR, get_dataset_dir
from muss.utils.helpers import add_dicts, args_str_to_dict
from muss.utils.resources import download_and_extract
from muss.preprocessors import GPT2BPEPreprocessor
from muss.preprocessing import apply_line_function_to_file
from muss.fairseq.main import get_language_from_dataset
from muss.text import truncate

from muss.simplify import simplify_sentences
from muss.utils.helpers import *

from muss.simplifiers import get_fairseq_simplifier, get_preprocessed_simplifier
from muss.preprocessors import get_preprocessors, get_preprocessor_by_name

%cd '/content/drive/MyDrive/Lexical_simp'


/content/drive/MyDrive/Baseline_MUSS_corrupt/muss
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/Baseline_MUSS_corrupt/muss
  Cloning https://github.com/feralvam/easse.git to /tmp/pip-install-kqe7huaj/easse_ca61f4f235b94fd7a233a7c1c12414bc
  Running command git clone -q https://github.com/feralvam/easse.git /tmp/pip-install-kqe7huaj/easse_ca61f4f235b94fd7a233a7c1c12414bc
  Cloning https://github.com/kpu/kenlm.git to /tmp/pip-install-kqe7huaj/kenlm_6d1869cace9f4e4883fda835ab721d32
  Running command git clone -q https://github.com/kpu/kenlm.git /tmp/pip-install-kqe7huaj/kenlm_6d1869cace9f4e4883fda835ab721d32
     |████████████████████████████████| 181 kB 15.5 MB/s 
     |████████████████████████████████| 10.4 MB 99.5 MB/s 
     |████████████████████████████████| 50 kB 8.0 MB/s 
     |████████████████████████████████| 1.7 MB 75.5 MB/s 
     |████████████████████████████████| 28.4 MB 411 kB/s 
     

/content/drive/MyDrive/Lexical_simp


In [4]:
Dataset = namedtuple('Dataset', 'name, train, test')
Model = namedtuple('Model', 'type, name, dimension, corpus, model')

In [5]:
# !wget https://www.inf.uni-hamburg.de/en/inst/ab/lt/resources/data/complex-word-identification-dataset/cwishareddataset.zip

In [6]:
# !unzip cwishareddataset.zip

In [7]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 200)
MAIN_PATH_DATASET = "/content/drive/MyDrive/Lexical_simp/english/"
genres = ['Wikipedia', 'WikiNews', 'News']
datasets = ['Train', 'Dev']
columns = ['id', 'sentence', "start", "end", "target", 
           "nat", "non_nat", "nat_marked", "non_nat_marked", "binary", "prob"]


datasets = [Dataset('Wikipedia', 'Train', 'Dev'),
            Dataset('WikiNews', 'Train', 'Dev'),
            Dataset('News', 'Train', 'Dev')]

feature_categories = []

def load_df(path):
    df = pd.read_csv(path, header=None, sep = "\t")
    # print(df)
    df.columns = columns
    return df


datasets = [Dataset(d.name, load_df(MAIN_PATH_DATASET + d.name + '_' + d.train + '.tsv'),
                            load_df(MAIN_PATH_DATASET + d.name + '_' + d.test + '.tsv')) for d in datasets]

# for ds in datasets:
#   temp_df = pd.read_csv(MAIN_PATH_DATASET+ds.name+'_'+ ds.test +'.tsv', header=None, sep = "\t" )
#   print(temp_df.shape)

In [8]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove.6B.zip -d embeddings

In [9]:
### Load embedding model 


from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

MAIN_PATH = 'embeddings/'


glove_models = []

glove_defs = [ Model('glove', 'glove.6B.300d.txt', 300, 'wikipedia+gigaword5', None)]
              
for model in glove_defs:
    glove_file = MAIN_PATH + model.name
    tmp_file = get_tmpfile(model.name + '-temp')
    glove2word2vec(glove_file, tmp_file)
    vecs = KeyedVectors.load_word2vec_format(tmp_file)
    glove_models.append(Model(model.type, model.name, model.dimension, model.corpus, vecs))
    print('load model : {}'.format(model.name))
    
print(glove_models)

load model : glove.6B.300d.txt
[Model(type='glove', name='glove.6B.300d.txt', dimension=300, corpus='wikipedia+gigaword5', model=<gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x7fdf256790d0>)]


In [10]:
dataframe = datasets[0].train[0:30]

def overlaps(start1, end1, start2, end2):
    return bool(range(max(start1, start2), min(end1, end2)+1))

def extract_ngrams_group(group):
    targets = zip(group['target'].values.tolist(), group['start'].values.tolist(),
                 group['end'].values.tolist(), group['binary'].values.tolist())
    for word, start, end, binary in targets:
        tokens = word.split()
        if len(tokens)>1:
            olap_words = [(w, b) for w, s, e, b in targets if overlaps(start, end, s, e)]
            
    
grouped = dataframe.groupby('sentence').apply(lambda group : extract_ngrams_group(group))

In [11]:
dataframe

,id,sentence,start,end,target,nat,non_nat,nat_marked,non_nat_marked,binary,prob
0,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",0,8,Normally,10,10,0,1,1,0.05
1,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",28,34,passed,10,10,0,1,1,0.05
2,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",15,19,land,10,10,0,0,0,0.00
3,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",43,49,future,10,10,1,0,1,0.05
4,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",43,61,future generations,10,10,1,2,1,0.15
5,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",50,61,generations,10,10,3,2,1,0.25
6,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",76,86,recognizes,10,10,2,4,1,0.30
7,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",91,100,community,10,10,0,0,0,0.00
8,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",104,115,traditional,10,10,1,3,1,0.20
9,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",104,142,traditional connection to that country,10,10,0,0,0,0.00


In [12]:
wordlist_lowercased = set(i.lower() for i in brown.words())
print (len(wordlist_lowercased))

49815


In [13]:
tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                      if unicodedata.category(chr(i)).startswith('P'))

def remove_punctuation(text):
    return text.translate(tbl)

@lru_cache(maxsize=None)
def all_tokens_with_index(context):
    curr_pos = 0
    targets = []
    j = 0
    w = 0
    curr_split = ''
    ctx_split = context.split()
    whitespaces = re.findall('\s+', context)
    num_whitespaces = [len(token) for token in whitespaces]
    num_whitespaces.append(1)
    tokens = word_tokenize(context)
    tokens = ['"' if token not in context else token for token in tokens]
    for index, token in enumerate(tokens, 1):
        targets.append((token, index, curr_pos, (curr_pos + len(token))))
        curr_pos += len(token)
        curr_split += token
        if ctx_split[j] == curr_split:
            curr_pos += num_whitespaces[w]
            j += 1
            w += 1
            curr_split = ''
    return [val for val in targets if val[0] != '"']

def build_vocabulary(sentences, embedding_model, dimension):
    all_words = [tpl[0] for sentence in sentences for tpl in sentence['seq']] + list(wordlist_lowercased)
    print('# Words : {}'.format(len(all_words)))
    counter = Counter(all_words)
    vocab_size = len(counter) + 1
    print('# Vocab : {}'.format(vocab_size))
    print('# embeding model  : {}'.format(len(embedding_model.vocab)))   
    word2index = {word : index for index, (word, count) in enumerate(counter.most_common(), 1)}
    index2word = {index : word for word, index in word2index.items()}
    # +1 required for pad token
    embedding_matrix = np.zeros(((vocab_size), dimension))
    missing_embed_words = []
    i_ = 0
    for word, index in word2index.items():
        if word in embedding_model.vocab:
            embedding = embedding_model[word]
        else:
             i_ +=1
             continue
        embedding_matrix[index] = embedding
    missing_embed_count = len(missing_embed_words)
    print('# Words missing embedding : {}'.format(missing_embed_count))
    print('Embedding shape : {}'.format(embedding_matrix.shape))
    print("i: ", i_ )
    return word2index, index2word, embedding_matrix

def forward_transformation(dataframe, lowercase = True, filter_punc = True, filtering = "a132"):
    grouped = dataframe.groupby('sentence').apply(lambda row : 
                        {'sent_id' : list(set(row['sent_id']))[0],
                         'sentence' : list(set(row['sentence']))[0], 
                         'tags': [tag for tag in zip(row['target'], 
                            row['start'], row['end'], row['binary'], row['prob'])]})
    sentences = []
    for vals in grouped:
        sent_id = vals['sent_id']
        sentence = vals['sentence']
        tags = vals['tags']
        tags_without_labels = [(word, start, end) for word, start, end, binary, prob in tags]
        all_tokens = all_tokens_with_index(sentence)
        sent_repr = [(word, start, end, tags[tags_without_labels.index((word, start, end))][3],
                     tags[tags_without_labels.index((word, start, end))][4])
           if (word, start, end) in tags_without_labels 
          else (word, start, end, 0, 0.0) for word, index, start, end in all_tokens]
        if lowercase:
            sent_repr = [(word.lower(), start, end, binary, prob) 
                         for word, start, end, binary, prob in sent_repr]
        if filter_punc:
            sent_repr = list(filter(lambda vals : remove_punctuation(vals[0]), sent_repr))
        if filtering:
            sent_repr = list(filter(lambda vals : vals[0] != "'s", sent_repr))
            sent_repr = list(filter(lambda vals : vals[0] != "``", sent_repr))
        sentences.append({'sent_id' : sent_id, 'sentence' : sentence, 'seq' : sent_repr})
    return sentences

def split_sentence_seqs(sentences):
    words, start_end, binary, prob = [], [], [] ,[]
    for sent in sentences:
        sequence = sent['seq']
        curr_w, curr_se, curr_b, curr_p = map(list, zip(*[(vals[0], 
            (vals[1], vals[2]), vals[3], vals[4]) for vals in sequence]))
        words.append(curr_w)
        start_end.append(curr_se)
        binary.append(curr_b)
        prob.append(curr_p)
    return words, start_end, binary, prob

In [14]:
datasets.append(Dataset('train_all_test_wiki', 
        datasets[0].train.append(datasets[1].train).append(datasets[2].train), datasets[0].test))

In [15]:
# Append train and test set
dataset_sel = datasets[3]
train_num_rows = dataset_sel.train.shape[0]
train_num_sents = len(list(set(dataset_sel.train.sentence.values.tolist())))

test_num_rows = dataset_sel.test.shape[0]
test_num_sents = len(list(set(dataset_sel.test.sentence.values.tolist())))

dataset = dataset_sel.train.append(dataset_sel.test)
dataset['sent_id'] = dataset.groupby('sentence').ngroup()
dataset_num_rows = dataset.shape[0]
dataset_num_sents = len(list(set(dataset.sentence.values.tolist())))

print('# Rows train : {}'.format(train_num_rows))
print('# Rows test : {}'.format(test_num_rows))
print('# Rows dataset : {}'.format(dataset_num_rows))

print('# Sents train : {}'.format(train_num_sents))
print('# Sents test : {}'.format(test_num_sents))
print('# Sents dataset : {}'.format(dataset_num_sents))

# Rows train : 27299
# Rows test : 694
# Rows dataset : 27993
# Sents train : 1988
# Sents test : 53
# Sents dataset : 2041


In [16]:
dataset

,id,sentence,start,end,target,nat,non_nat,nat_marked,non_nat_marked,binary,prob,sent_id
0,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",0,8,Normally,10,10,0,1,1,0.05,1347
1,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",28,34,passed,10,10,0,1,1,0.05,1347
2,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",15,19,land,10,10,0,0,0,0.00,1347
3,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",43,49,future,10,10,1,0,1,0.05,1347
4,3XU9MCX6VODXPI3L8I02CM94TFB2R7,"Normally , the land will be passed down to future generations in a way that recognizes the community 's traditional connection to that country .",43,61,future generations,10,10,1,2,1,0.15,1347
...,...,...,...,...,...,...,...,...,...,...,...,...
689,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of bad Karma and will bring a person closer to the true knowledge by purifying his mind .,100,109,knowledge,10,10,0,2,1,0.10,929
690,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of bad Karma and will bring a person closer to the true knowledge by purifying his mind .,74,80,person,10,10,0,0,0,0.00,929
691,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of bad Karma and will bring a person closer to the true knowledge by purifying his mind .,95,99,true,10,10,0,0,0,0.00,929
692,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of bad Karma and will bring a person closer to the true knowledge by purifying his mind .,113,122,purifying,10,10,7,8,1,0.75,929


In [17]:
sentences = forward_transformation(dataset)
train_sentences = sentences[:train_num_sents]
test_sentences = sentences[train_num_sents:]

In [18]:
words, start_end, binary, prob = split_sentence_seqs(sentences)

In [19]:
sentence_lens = [len(sent) for sent in words]
print(sentence_lens)

[37, 52, 34, 25, 19, 46, 25, 23, 19, 21, 34, 27, 34, 20, 20, 16, 14, 13, 17, 19, 18, 9, 20, 7, 16, 19, 6, 17, 28, 17, 10, 6, 30, 9, 24, 14, 11, 23, 12, 10, 15, 22, 36, 21, 22, 20, 30, 13, 22, 17, 8, 16, 21, 26, 23, 37, 45, 9, 37, 16, 27, 24, 25, 24, 21, 26, 15, 13, 25, 33, 10, 69, 48, 16, 20, 23, 73, 23, 27, 25, 40, 33, 12, 32, 21, 25, 10, 12, 9, 22, 13, 11, 13, 27, 8, 23, 12, 25, 16, 20, 14, 8, 16, 9, 19, 9, 17, 28, 17, 17, 18, 13, 22, 14, 18, 10, 14, 13, 17, 17, 8, 27, 20, 16, 15, 21, 17, 50, 23, 20, 21, 21, 9, 14, 26, 21, 35, 13, 20, 12, 29, 60, 23, 19, 21, 15, 22, 16, 28, 15, 14, 12, 29, 12, 25, 20, 13, 26, 23, 15, 30, 8, 13, 23, 8, 16, 16, 13, 12, 10, 10, 10, 20, 23, 9, 27, 18, 51, 28, 9, 36, 15, 20, 27, 14, 75, 22, 34, 19, 23, 13, 11, 29, 43, 22, 18, 33, 18, 14, 8, 48, 24, 48, 43, 29, 32, 19, 24, 47, 18, 31, 51, 16, 18, 13, 15, 26, 31, 25, 18, 22, 8, 34, 24, 40, 15, 10, 11, 8, 26, 19, 11, 31, 15, 14, 16, 23, 15, 43, 24, 27, 29, 32, 10, 42, 12, 25, 8, 22, 17, 14, 30, 46, 24, 16, 7

In [20]:
embedding_model = glove_models[0].model
dimension = embedding_model.vector_size
word2index, index2word, embedding = build_vocabulary(sentences, embedding_model, dimension)


# Words : 96437
# Vocab : 52455
# embeding model  : 400000
# Words missing embedding : 0
Embedding shape : (52455, 300)
i:  9834


Padding the input sequences and get the binaries labels

In [21]:
words_with_indices = [[word2index[word] for word in sent] for sent in words]
sent_lens = [len(sentence['seq']) for sentence in sentences]
sent_max_length = np.max(sent_lens)
print('Max length sentence : {}'.format(sent_max_length))

from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

words_padded = pad_sequences(maxlen=sent_max_length, sequences=words_with_indices, padding="post", value=0)
binary_padded = pad_sequences(maxlen=sent_max_length, sequences=binary, padding="post", value=0)
prob_padded = pad_sequences(maxlen=sent_max_length, sequences=prob, padding="post", value=0, dtype="float")

binary_padded_categorical = [to_categorical(clazz, num_classes=2) for clazz in binary_padded]

Max length sentence : 103


In [22]:
# (1) Training set
train_words_padded = words_padded[:train_num_sents]
train_binary_padded = binary_padded[:train_num_sents]
train_binary_padded_categorical = binary_padded_categorical[:train_num_sents]
train_prob_padded = prob_padded[:train_num_sents]
train_start_end = start_end[:train_num_sents]

# (2) Test set
test_words_padded = words_padded[train_num_sents:]
test_binary_padded = binary_padded[train_num_sents:]
test_binary_padded_categorical = binary_padded_categorical[train_num_sents:]
test_prob_padded = prob_padded[train_num_sents:]
test_start_end = start_end[train_num_sents:]

print('Training set length : {}'.format(len(train_words_padded)))
print('Test set length : {}'.format(len(test_words_padded)))

Training set length : 1988
Test set length : 53


In [23]:
from sklearn.metrics import f1_score
import keras.callbacks
from keras import backend as K
class Metrics(keras.callbacks.Callback):
    def __init__(self, validation_data):
        self.f1_scores = []
        self.validation_data = validation_data
        
    def on_epoch_end(self, batch, logs={}):
        predict = np.asarray(self.model.predict(self.validation_data[0]))
        targ = self.validation_data[1]
        targ = np.array(targ)
        shape = targ.shape
        targ = targ.reshape((shape[0]*shape[1], shape[2]))
        targ = np.argmax(targ, axis = 1)
        predict = predict.reshape((shape[0]*shape[1]), shape[2])
        predict = np.argmax(predict, axis = 1)
        self.f1s=f1_score(targ, predict)
        print("\nF1 Score:")
        print(f1_score(targ, np.ones(shape[0]*shape[1])))
        self.f1_scores.append(self.f1s)
        return

In [24]:
from keras.models import Model, Input
from keras import backend as K
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

vocab_size = embedding.shape[0]
dimension = embedding.shape[1]

np.set_printoptions(threshold=np.inf)

in_seq = Input(shape=(sent_max_length,))
embed = Embedding(input_dim=vocab_size, output_dim=dimension, \
                  weights=[embedding], input_length=sent_max_length)(in_seq)
drop = Dropout(0.1)(embed)
lstm = Bidirectional(LSTM(units=150, return_sequences=True, recurrent_dropout=0.1))(drop)
out = TimeDistributed(Dense(2, activation="softmax"))(lstm) 

model = Model(in_seq, out)

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=["accuracy"])
model.summary()

metrics = Metrics((test_words_padded, np.array(test_binary_padded_categorical)))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 103)]             0         
                                                                 
 embedding (Embedding)       (None, 103, 300)          15736500  
                                                                 
 dropout (Dropout)           (None, 103, 300)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 103, 300)         541200    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 103, 2)           602       
 ibuted)                                                         
                                                                 
Total params: 16,278,302
Trainable params: 16,278,302
Non-tra

In [25]:
# history = model.fit(train_words_padded, np.array(train_binary_padded_categorical), batch_size=10, 
#                     epochs=3, validation_data = (test_words_padded, np.array(test_binary_padded_categorical)), 
#                     verbose=1, callbacks=[metrics])

In [26]:
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()

In [27]:
# import seaborn as sns
# sns.set_style("darkgrid")
# epoch_f1s = plt.plot(metrics.f1_scores)
# plt.show()


In [28]:
model_save_name = 'model_CWI_full.h5'
path_dir = F"/content/drive/MyDrive/Lexical_simp/{model_save_name}"

In [29]:
from keras.models import load_model

model.save(path_dir)  # creates a HDF5 file 'model_CWI_full.h5'

In [30]:
from keras.models import load_model

model_cwi = load_model(path_dir)

Now, let´s define some useful functions in order to use the CWI with some out of samples sentences

In [31]:
stop_words_ = set(stopwords.words('english'))
def cleaner(word):
  #Remove links
  word = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', 
                '', word, flags=re.MULTILINE)
  word = re.sub('[\W]', ' ', word)
  word = re.sub('[^a-zA-Z]', ' ', word)
  return word.lower().strip()

In [32]:
def process_input(input_text):
  input_text = cleaner(input_text)
  clean_text = []
  index_list =[]
  input_token = []
  index_list_zipf = []
  for i, word in enumerate(input_text.split()):
    if word in word2index:
      clean_text.append(word)
      input_token.append(word2index[word])
    else:
      index_list.append(i)
  input_padded = pad_sequences(maxlen=sent_max_length, sequences=[input_token], padding="post", value=0)
  return input_padded, index_list, len(clean_text)

In [33]:
def complete_missing_word(pred_binary, index_list, len_list):
  list_cwi_predictions = list(pred_binary[0][:len_list])
  for i in index_list:
    list_cwi_predictions.insert(i, 0)
  return list_cwi_predictions

#Second part: The Candidates generation and selection using BERT

In [34]:
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

bert_model = 'bert-large-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_model)
model = BertForMaskedLM.from_pretrained(bert_model)
model.eval()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementw

In [36]:
!pip install wordfreq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56.8 MB 17 kB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 


In [37]:
from wordfreq import zipf_frequency
zipf_frequency('stop', 'en')


5.52

In [38]:
from wordfreq import zipf_frequency
zipf_frequency('thwart', 'en')

3.06

# Now the function to get the candidates out of BERT (MLM):

In [39]:

def get_bert_candidates(input_text, list_cwi_predictions, numb_predictions_displayed = 10):
  list_candidates_bert = []
  for word,pred  in zip(input_text.split(), list_cwi_predictions):
    if (pred and (pos_tag([word])[0][1] in ['NNS', 'NN', 'VBP', 'RB', 'VBG','VBD' ]))  or (zipf_frequency(word, 'en')) <3.1:
      replace_word_mask = input_text.replace(word, '[MASK]')
      text = f'[CLS]{replace_word_mask} [SEP] {input_text} [SEP] '
      tokenized_text = tokenizer.tokenize(text)
      masked_index = [i for i, x in enumerate(tokenized_text) if x == '[MASK]'][0]
      indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
      segments_ids = [0]*len(tokenized_text)
      tokens_tensor = torch.tensor([indexed_tokens])
      segments_tensors = torch.tensor([segments_ids])
      # Predict all tokens
      with torch.no_grad():
          outputs = model(tokens_tensor, token_type_ids=segments_tensors)
          predictions = outputs[0][0][masked_index]
      predicted_ids = torch.argsort(predictions, descending=True)[:numb_predictions_displayed]
      predicted_tokens = tokenizer.convert_ids_to_tokens(list(predicted_ids))
      list_candidates_bert.append((word, predicted_tokens))
  return list_candidates_bert
  

#Simplifying new sentences:


In [ ]:
list_texts = [ 
 'The Risk That Students Could Arrive at School With the Coronavirus As schools grapple with how to reopen, new estimates show that large parts of the country would probably see infected students if classrooms opened now.',
 'How a photograph of a young man cradling his dying friend sent me on a journey across India.',
 'Pro-democracy parties, which had hoped to ride widespread discontent to big gains, saw the yearlong delay as an attempt to thwart them.',
 'Night after night, calm gave way to chaos. See what happened between the protesters and the federal agents.',
 'Contact Tracing Is Failing in Many States. Here is Why. Inadequate testing and protracted delays in producing results have crippled tracking and hampered efforts to contain major outbreaks.',
 'After an initial decrease in the youth detention population, the rate of release has slowed, and the gap between white youth and Black youth has grown.'
 'A laboratory experiment hints at some of the ways the virus might elude antibody treatments. Combining therapies could help, experts said.',
 'Though I may not be here with you, I urge you to answer the highest calling of your heart and stand up for what you truly believe.',
 'The research does not prove that infected children are contagious, but it should influence the debate about reopening schools, some experts said.',
 'Dropping antibody counts are not a sign that our immune system is failing against the coronavirus, nor an omen that we can not develop a viable vaccine.',
 'The Senate majority leader has said he will not approve a stimulus package without a “liability shield,” but top White House officials say they do not see it as essential.',
 'Campaign efforts to refocus come as the president continues to push divisive messages that have frustrated his own party.'
] 

In [ ]:
ls_result = []
ls_replacement = []

counter = 0
for input_text in list_texts:
  counter += 1 
  new_text = input_text
  input_padded, index_list, len_list = process_input(input_text)
  pred_cwi = model_cwi.predict(input_padded)
  pred_cwi_binary = np.argmax(pred_cwi, axis = 2)
  complete_cwi_predictions = complete_missing_word(pred_cwi_binary, index_list, len_list)
  bert_candidates = get_bert_candidates(input_text, complete_cwi_predictions)
  ls_replace_temp = []
  try:
    for word_to_replace, l_candidates in bert_candidates:
      tuples_word_zipf = []
      for w in l_candidates:
        if w.isalpha():
          tuples_word_zipf.append((w, zipf_frequency(w, 'en')))
      tuples_word_zipf = sorted(tuples_word_zipf, key = lambda x: x[1], reverse=True)
      new_text = re.sub(word_to_replace, tuples_word_zipf[0][0], new_text)

      ls_replace_temp.append([word_to_replace,tuples_word_zipf[0][0]])
    
    ls_replacement.append(ls_replace_temp)
    ls_result.append([input_text,new_text])  # Input is the complex, new_text is the simplified one 
    print(input_text) 
    print(tuples_word_zipf)
    print(new_text)
  except:
    pass
  if counter%1000 == 0:
    print(counter,'counter')
    break

How a photograph of a young man cradling his dying friend sent me on a journey across India.
[('took', 5.51), ('sent', 5.16), ('received', 5.16), ('send', 5.11), ('led', 5.11), ('carried', 4.77), ('sending', 4.61), ('directed', 4.6), ('sends', 4.15), ('dispatched', 3.61)]
How a photograph of a young man and his dying friend took me on a journey across India.
Pro-democracy parties, which had hoped to ride widespread discontent to big gains, saw the yearlong delay as an attempt to thwart them.
[('stop', 5.52), ('prevent', 4.79), ('counter', 4.56), ('defeat', 4.51), ('delay', 4.4), ('halt', 3.93), ('stall', 3.79), ('undermine', 3.65), ('disrupt', 3.59)]
other party which have hoped to ride widespread discontent to big results see the year delay as an attempt to stop them.
Night after night, calm gave way to chaos. See what happened between the protesters and the federal agents.
[('agent', 4.81), ('agents', 4.55)]
day after day calm give in to chaos See what was between the people and the 

ERROR: Invalid requirement: '==2.0.0'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached en_core_web_md-2.3.1-py3-none-any.whl
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


OSError: ignored

In [40]:
import re
def NE_extraction(complex_sentence):
  control_tokens = ["<NEXT_NE> ","<NEXT_DIFFICULT_WORD>","<SENT_START>"]

  def list_duplicates_of(seq,item):
    start_at = -1
    locs = []
    while True:
        try:
            loc = seq.index(item,start_at+1)
        except ValueError:
            break
        else:
            locs.append(loc)
            start_at = loc
    return locs

  indice_of_NE = list_duplicates_of(complex_sentence,control_tokens[0])
  indice_of_diff_NE = list_duplicates_of(complex_sentence,control_tokens[1])
  indice_of_start = list_duplicates_of(complex_sentence,control_tokens[2])

  NE_list = {}
  # if 
  if not indice_of_diff_NE:
    # print('none')
    temp_list = []
    for i,j in enumerate(indice_of_NE[:-1]):
      temp_list.append(complex_sentence[indice_of_NE[i]+len(control_tokens[0]):indice_of_NE[i+1]-1])
    temp_list.append(complex_sentence[indice_of_NE[-1]+len(control_tokens[0]):indice_of_start[0]-1])
    NE_list['NE_simple'] = temp_list
    NE_list['NE_difficult'] = []
          
  else:
    # print('difficult word exist')
    temp_list = []
    temp_list2 = []
    for i,j in enumerate(indice_of_NE[:-1]):
      temp_list.append(complex_sentence[indice_of_NE[i]+len(control_tokens[0]):indice_of_NE[i+1]-1])
    temp_list.append(complex_sentence[indice_of_NE[-1]+len(control_tokens[0]):indice_of_diff_NE[0]-1])
    NE_list['NE_simple'] = temp_list
    for i,j in enumerate(indice_of_diff_NE[:-1]):
      temp_list2.append(complex_sentence[indice_of_diff_NE[i]+len(control_tokens[1]):indice_of_diff_NE[i+1]-1])
    temp_list2.append(complex_sentence[indice_of_diff_NE[-1]+len(control_tokens[1]):indice_of_start[0]-1])
    # print(NE_list)
    NE_list['NE_difficult'] = temp_list2
  return NE_list, complex_sentence[indice_of_start[0]+len(control_tokens[2])+1:]

In [ ]:
import csv

wiki = []
wiki1 = []
with open('/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/0913_ABCD_NER_wikilarge/train.complex') as f:
  csv_reader = csv.reader(f)
  for row in csv_reader:
    wiki.append(row[0])

for i,j in enumerate(wiki):
  try: 
    wiki1.append(NE_extraction(j)[1])
  except:
    # print('err')
    pass
print(wiki[:100])
print(wiki1[:100])
print(len(wiki),len(wiki1))

[' <NEXT_NE> James Edward <NEXT_NE> Anna Austen <NEXT_NE> 1814 <NEXT_DIFFICULT_WORD> manuscript <NEXT_DIFFICULT_WORD> evidence <SENT_START> There is manuscript evidence that Austen continued to work on these pieces as late as the period 1809 â "11', " <NEXT_NE> Psalms of Thomas <NEXT_NE> Säve-Söderberg <NEXT_NE> Mani's Psalms <NEXT_NE> Mandaean <NEXT_DIFFICULT_WORD> comparative <NEXT_DIFFICULT_WORD> analysis <SENT_START> In a remarkable comparative analysis", ' <NEXT_NE> Underworld <NEXT_NE> Persephone <NEXT_NE> Demeter <NEXT_NE> Hades <NEXT_DIFFICULT_WORD> released <NEXT_DIFFICULT_WORD> tricked <SENT_START> Before Persephone was released to Hermes', ' <NEXT_NE> the Tribune de Genève <NEXT_NE> 1 February 1879 <NEXT_NE> James T. Bates <NEXT_NE> about 187', ' <NEXT_NE> Oxford English Dictionary <NEXT_NE> Old English <NEXT_NE> 1933 <NEXT_DIFFICULT_WORD> agricultural <NEXT_DIFFICULT_WORD> reaping <NEXT_DIFFICULT_WORD> crops <NEXT_DIFFICULT_WORD> reaping <NEXT_DIFFICULT_WORD> i.e. <SENT_STA

In [ ]:
ls_result = []
ls_replacement = []

counter = 0
for input_text in wiki1:
  counter += 1 
  new_text = input_text
  input_padded, index_list, len_list = process_input(input_text)
  pred_cwi = model_cwi.predict(input_padded)
  pred_cwi_binary = np.argmax(pred_cwi, axis = 2)
  complete_cwi_predictions = complete_missing_word(pred_cwi_binary, index_list, len_list)
  bert_candidates = get_bert_candidates(input_text, complete_cwi_predictions)
  ls_replace_temp = []
  try:
    for word_to_replace, l_candidates in bert_candidates:
      tuples_word_zipf = []
      for w in l_candidates:
        if w.isalpha():
          tuples_word_zipf.append((w, zipf_frequency(w, 'en')))
      tuples_word_zipf = sorted(tuples_word_zipf, key = lambda x: x[1], reverse=True)
      new_text = re.sub(word_to_replace, tuples_word_zipf[0][0], new_text)
      ls_replace_temp.append([word_to_replace,tuples_word_zipf[0][0]])
    
    ls_replacement.append(ls_replace_temp)
    ls_result.append([input_text,new_text])  # Input is the complex, new_text is the simplified one 
      
  except:
    pass
  if counter%10 == 0:
    print(counter,'counter')
    break
  # break
  # break
  # print("Original text: ", input_text )
  # print("Simplified text:", new_text, "\n")

There is manuscript evidence that Austen continued to work on these pieces as late as the period 1809 â "11
[('no', 6.35), ('some', 6.2), ('film', 5.2), ('written', 5.09), ('document', 4.5), ('documentary', 4.31), ('literary', 4.31), ('considerable', 4.25), ('manuscript', 3.92), ('unpublished', 3.41)]
There is no evidence that Austen continued to work on these pieces as late as the period 1809 â "11
Original text:  There is manuscript evidence that Austen continued to work on these pieces as late as the period 1809 â "11
Simplified text: There is no evidence that Austen continued to work on these pieces as late as the period 1809 â "11 

In a remarkable comparative analysis
[('study', 5.26), ('review', 5.12), ('analysis', 4.91), ('comparison', 4.51)]
In a remarkable comparative study
Original text:  In a remarkable comparative analysis
Simplified text: In a remarkable comparative study 

Before Persephone was released to Hermes
[('i', 7.09), ('it', 6.95), ('he', 6.69), ('she', 6.26), (

In [ ]:
ls_result[:100]

In [ ]:
ls_replacement[:100]

In [ ]:
import pickle
with open('/content/drive/MyDrive/Lexical_simp/lexical_train_wiki.pkl','wb') as f:
  a = [ls_result,ls_replacement]
  pickle.dump(a,f)

In [ ]:
with open('/content/drive/MyDrive/Lexical_simp/lexical_train_wiki.pkl','rb') as f:
  # a = [ls_result,ls_replacement]
  data = pickle.load(f)
ls_result,ls_replacement = data[0],data[1]

In [ ]:
# import spacy
!python -m spacy download en_core_web_md
import spacy 
nlp = spacy.load('en_core_web_md')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached en_core_web_md-2.3.1-py3-none-any.whl
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


OSError: ignored

if the cosine similarity is too low, I will replace the word back to the original sentence and delete the ls_replacment word
 
 

In [ ]:
count = 0



for i,sen in enumerate(ls_replacement):
  for k,words in enumerate(sen):
    count += 1 
    if count % 10000 == 0:
      print(count)
    token1,token2 = nlp(words[0]),nlp(words[1])

    if token1.similarity(token2) <0.25:
      # print(ls_result[i])
      ls_result[i][1] = ls_result[i][1].replace(words[1],words[0])
      ls_replacement[i][k] = None
      # print(words[0],words[1],token1.similarity(token2))

  # break


In [ ]:
# count = 0
# for i in ls_replacement:
#   for j in i:
#     count += 1
# print(count)

# 209001
import pickle
with open('/content/drive/MyDrive/Lexical_simp/lexical_train_wiki_filtered.pkl','wb') as f:
  a = [ls_result,ls_replacement]
  pickle.dump(a,f)

In [ ]:
count = 0
for i in ls_replacement:
  for j in i:
    count += 1
print(count)

In [ ]:
ls_result

# Create Asset dataset with NE prefix 


In [44]:
import csv
%cd /content/drive/MyDrive/Baseline_MUSS_corrupt/muss
# asset = read_lines('/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/valid.complex')
asset_test = read_lines('/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/test.complex')

/content/drive/MyDrive/Baseline_MUSS_corrupt/muss


In [45]:
import csv

# asset_test = read_lines(asset)
asset_test1 = []

for i,j in enumerate(asset_test):
  try: 
    asset_test1.append(NE_extraction(j)[1])
  except:
    # print('err')
    pass
print(asset_test[:100])
print(asset_test1[:100])
print(len(asset_test),len(asset_test1))

['One side of the armed conflicts is composed mainly of the Sudanese military and the Janjaweed, a Sudanese militia group recruited mostly from the Afro-Arab Abbala tribes of the northern Rizeigat region in Sudan.', "Jeddah is the principal gateway to Mecca, Islam's holiest city, which able-bodied Muslims are required to visit at least once in their lifetime.", 'The Great Dark Spot is thought to represent a hole in the methane cloud deck of Neptune.', 'His next work, Saturday, follows an especially eventful day in the life of a successful neurosurgeon.', 'The tarantula, the trickster character, spun a black cord and, attaching it to the ball, crawled away fast to the east, pulling on the cord with all his strength.', 'There he died six weeks later, on 13 January 888.', 'They are culturally akin to the coastal peoples of Papua New Guinea.', 'Since 2000, the recipient of the Kate Greenaway Medal has also been presented with the Colin Mears Award to the value of £5000.', 'Following the dr

In [50]:
def find_NE(data):
  ls_result = []
  ls_replacement = []

  counter = 0
  for input_text in data:
    counter += 1 
    new_text = input_text
    input_padded, index_list, len_list = process_input(input_text)
    pred_cwi = model_cwi.predict(input_padded)
    pred_cwi_binary = np.argmax(pred_cwi, axis = 2)
    complete_cwi_predictions = complete_missing_word(pred_cwi_binary, index_list, len_list)
    bert_candidates = get_bert_candidates(input_text, complete_cwi_predictions)
    ls_replace_temp = []
    try:
      for word_to_replace, l_candidates in bert_candidates:
        tuples_word_zipf = []
        for w in l_candidates:
          if w.isalpha():
            tuples_word_zipf.append((w, zipf_frequency(w, 'en')))
        tuples_word_zipf = sorted(tuples_word_zipf, key = lambda x: x[1], reverse=True)
        new_text = re.sub(word_to_replace, tuples_word_zipf[0][0], new_text) 
        ls_replace_temp.append([word_to_replace,tuples_word_zipf[0][0]])
      
      ls_replacement.append(ls_replace_temp)
      ls_result.append(new_text)  # Input is the complex, new_text is the simplified one 

    except:
      ls_replacement.append([])
      ls_result.append(new_text)
    if counter%1000 == 0:
      print(counter,'counter')
  return ls_result, ls_replacement

In [ ]:
asset_result, asset_replacement = find_NE(asset)
len(asset_result)

1000 counter
2000 counter


2000

In [51]:
asset_result_test, asset_replacement_test = find_NE(asset_test)

In [48]:
input = asset_result

train_input = []
train_target = []
count = 0
for i,j in enumerate(input):
  if count > -1:
    if asset_replacement[i] is not None:
      NER = [k[0] for k in asset_replacement[i] if k is not None]
      if NER:
        if len(NER) >1:
          sentence_w_control_token = ' <NEXT_NE> '+' <NEXT_NE> '.join(NER) + " <SENT_START> " + asset_test[i]
          train_input.append(sentence_w_control_token)
          train_target.append(asset_result[i])
          # print(sentence_w_control_token)
        else:
          sentence_w_control_token = ' <NEXT_NE> '+' <NEXT_NE> '.join(NER) + " <SENT_START> " + asset_test[i]
          train_input.append(sentence_w_control_token)
          train_target.append(asset_result[i])
      else:
          sentence_w_control_token = ' <NEXT_NE> '+' <NEXT_NE> '.join(NER) + " <SENT_START> " + asset_test[i]
          train_input.append(sentence_w_control_token)
    else:
      sentence_w_control_token = ' <NEXT_NE> '+' <NEXT_NE> '.join(NER) + " <SENT_START> " + asset_test[i]
      train_input.append(sentence_w_control_token)

  count += 1 
train_input[:15]



NameError: ignored

In [54]:
input = asset_result_test

train_input_assettest = []
train_target = []
count = 0
for i,j in enumerate(input):
  if count > -1:
    if asset_replacement_test[i] is not None:
      NER = [k[0] for k in asset_replacement_test[i] if k is not None]
      if NER:
        if len(NER) >1:
          sentence_w_control_token = ' <NEXT_NE> '+' <NEXT_NE> '.join(NER) + " <SENT_START> " + j
          train_input_assettest.append(sentence_w_control_token)
          train_target.append(asset_result_test[i])
          # print(sentence_w_control_token)
        else:
          sentence_w_control_token = ' <NEXT_NE> '+' <NEXT_NE> '.join(NER) + " <SENT_START> " + j
          train_input_assettest.append(sentence_w_control_token)
          train_target.append(asset_result_test[i])
      else:
        sentence_w_control_token = ' <NEXT_NE> '+' <NEXT_NE> '.join(NER) + " <SENT_START> " + j
        train_input_assettest.append(sentence_w_control_token)
    else:
      sentence_w_control_token = ' <NEXT_NE> '+' <NEXT_NE> '.join(NER) + " <SENT_START> " + j
      train_input_assettest.append(sentence_w_control_token)

  count += 1 
train_input_assettest[:15]

[' <NEXT_NE> mainly <NEXT_NE> Janjaweed, <NEXT_NE> militia <NEXT_NE> group <NEXT_NE> mostly <NEXT_NE> Abbala <NEXT_NE> Rizeigat <SENT_START> One side of the armed conflicts is composed mostly of the Sudanese military and the police a Sudanese military group recruited mostly from the Afro-Arab hill tribes of the northern central region in Sudan.',
 " <NEXT_NE> Jeddah <NEXT_NE> Islam's <NEXT_NE> holiest <SENT_START> it is the principal gateway to Mecca, the capital city, which able-bodied Muslims are required to visit at least once in their lifetime.",
 ' <NEXT_NE> deck <NEXT_NE> Neptune. <SENT_START> The Great Dark Spot is thought to represent a hole in the methane cloud top of earth',
 ' <NEXT_NE> work, <NEXT_NE> eventful <NEXT_NE> neurosurgeon. <SENT_START> His next work Saturday, follows an especially happy day in the life of a successful man',
 ' <NEXT_NE> tarantula, <NEXT_NE> trickster <NEXT_NE> character, <NEXT_NE> spun <NEXT_NE> cord <NEXT_NE> and, <NEXT_NE> away <NEXT_NE> fast <

In [ ]:
!pip install imohash
from muss.utils.helpers import *

In [55]:
from itertools import product
%cd '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/'


data_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset_ne2_simp/'

PHASES = ['test']
LANGUAGES = ['complex']
data_names = ['train_input_assettest']
# random.shuffle(train_input)
# x = [i for i in range(len(train_input))]
# random.shuffle(x)

count = 0
for phase, language in product(PHASES, LANGUAGES):
  # index = slice(0,-1500)
  # train_inputx = train_input[index]
  # train_targetx = train_target[index]
  # print(train_input[:10])
  print(phase+'.'+language)
  with open(data_dir+phase+'.'+language,'w') as f:
    for sen in globals()[data_names[count]]:
      if type(sen) == str:
        f.write("{}\n".format(sen))
  count += 1
f.close()

# read_lines(data_dir+phase+'.'+LANGUAGES[0])

/content/drive/MyDrive/Baseline_MUSS_corrupt/muss
test.complex


In [56]:
read_lines(data_dir+phase+'.'+LANGUAGES[0])[:10]
read_lines(data_dir+PHASES[0]+'.'+LANGUAGES[0])[:10]

[' <NEXT_NE> mainly <NEXT_NE> Janjaweed, <NEXT_NE> militia <NEXT_NE> group <NEXT_NE> mostly <NEXT_NE> Abbala <NEXT_NE> Rizeigat <SENT_START> One side of the armed conflicts is composed mostly of the Sudanese military and the police a Sudanese military group recruited mostly from the Afro-Arab hill tribes of the northern central region in Sudan.',
 " <NEXT_NE> Jeddah <NEXT_NE> Islam's <NEXT_NE> holiest <SENT_START> it is the principal gateway to Mecca, the capital city, which able-bodied Muslims are required to visit at least once in their lifetime.",
 ' <NEXT_NE> deck <NEXT_NE> Neptune. <SENT_START> The Great Dark Spot is thought to represent a hole in the methane cloud top of earth',
 ' <NEXT_NE> work, <NEXT_NE> eventful <NEXT_NE> neurosurgeon. <SENT_START> His next work Saturday, follows an especially happy day in the life of a successful man',
 ' <NEXT_NE> tarantula, <NEXT_NE> trickster <NEXT_NE> character, <NEXT_NE> spun <NEXT_NE> cord <NEXT_NE> and, <NEXT_NE> away <NEXT_NE> fast <

In [ ]:
len(read_lines(data_dir+PHASES[0]+'.'+LANGUAGES[0]))